In [3]:
import numpy as np
import scipy.misc
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input,decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor

In [13]:
def identity_block (X, f, filters, training=True, initializer= random_uniform):
    """ 
    X- input tensor of shape (m,n_H_prev,n_W_prev,n_C_prev)
    f-integers, specifying the shape of the middle CONV's window of the main path
    filters-- python list of integers, defining the number of filters in the CONV layers of the main path
    training- True, behave in training mode, False - behave in inference mode
    initializer - to set up the initial weights of a layer. Equals to random uniform initializer
    """

    #Retreive filters
    F1, F2, F3 = filters

    #Save the input value, you'll need this to add back to main path
    X_skip= X
    cache=[]

    #First component of main path
    X=Conv2D(filters=F1,kernel_size=1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)

    #Second Component of the main path
    X=Conv2D(filters=F2,kernel_size=(f,f),strides=(1,1),padding='same',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)
    X=Activation('relu')(X)

    #Third component of the main path
    X=Conv2D(filters=F3,kernel_size=1,strides=(1,1),padding='valid',kernel_initializer=initializer(seed=0))(X)
    X=BatchNormalization(axis=3)(X,training=training)

    #Final step
    X=Add()([X_skip,X])
    X=Activation('relu')(X)

    return X

In [14]:
np.random.seed(1)
X1 = np.ones((1, 4, 4, 3)) * -1
X2 = np.ones((1, 4, 4, 3)) * 1
X3 = np.ones((1, 4, 4, 3)) * 3

X = np.concatenate((X1, X2, X3), axis = 0).astype(np.float32)

A3 = identity_block(X, f=2, filters=[4, 4, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=False)
print('\033[1mWith training=False\033[0m\n')
A3np = A3.numpy()
print(np.around(A3.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))
resume = A3np[:,(0,-1),:,:].mean(axis = 3)
print(resume[1, 1, 0])

print('\n\033[1mWith training=True\033[0m\n')
np.random.seed(1)
A4 = identity_block(X, f=2, filters=[3, 3, 3],
                   initializer=lambda seed=0:constant(value=1),
                   training=True)
print(np.around(A4.numpy()[:,(0,-1),:,:].mean(axis = 3), 5))

With training=False

[[[  0.        0.        0.        0.     ]
  [  0.        0.        0.        0.     ]]

 [[192.71233 192.71233 192.71233  96.85616]
  [ 96.85616  96.85616  96.85616  48.92808]]

 [[578.13696 578.13696 578.13696 290.56848]
  [290.56848 290.56848 290.56848 146.78424]]]
96.85616

With training=True

[[[0.      0.      0.      0.     ]
  [0.      0.      0.      0.     ]]

 [[0.40739 0.40739 0.40739 0.40739]
  [0.40739 0.40739 0.40739 0.40739]]

 [[4.99991 4.99991 4.99991 3.25948]
  [3.25948 3.25948 3.25948 2.40739]]]
